# Text Generation Hugging Face Pipeline
* Notebook by Adam Lang
* Date: 12/3/2024

# Overview
* In this notebook I will demo how to implement a simple text generation pipeline in hugging face.

# Install Dependencies

In [1]:
!pip install -U transformers #upgrades
!pip install -U sentencepiece #upgrades
!pip install -U sacremoses #upgrades

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 23.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 19.4 MB/s eta 0:00:00


In [2]:
## imports - transformers
from transformers import pipeline
from transformers import set_seed
set_seed(42) #set seed for consistency


import pandas as pd
import numpy as np

# Setup Text Generation Pipeline
* The default text generator model is: `openai-community/gpt2 and revision 607a30d`
  * model card: https://huggingface.co/openai-community/gpt2

In [3]:
## lets use some dummy text data as an example
text = """
Dear Amazon, last week I ordered a new pair of alpine skis
from your online store in Seattle. Unfortunately when I opened
the package, I discovered that I had accidentally been sent a Snowboard instead!
"""

In [7]:
## generator setup
generator = pipeline("text-generation")

# response
response = "I am so sorry to hear that your order was mixed up"

# prompt --> replace new line chars with `.replace`
prompt = "user: " + text.replace("\n", " ") + " Customer Service Response: " + response

## get outputs from text generator
outputs= generator(prompt,
                   max_length=128, #change based on use case
                   )

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [8]:
## outputs
outputs

[{'generated_text': 'user:  Dear Amazon, last week I ordered a new pair of alpine skis from your online store in Seattle. Unfortunately when I opened the package, I discovered that I had accidentally been sent a Snowboard instead!  Customer Service Response: I am so sorry to hear that your order was mixed up with a snowboard shipment!  I am so sorry I purchased a pair of Alpaca in Amazon and would love to review both of them in the future to check their compatibility. Thank you for visiting the store and I look forward to seeing them next time. Thanks for your interest in my order!!!! I would appreciate a little'}]

Summary
* We can see that GPT-2 is hallucinating its answer as it doesn't have much context and is clearly regurgitating content from the internet corpus it was trained on.
* Let's see if we can improve this!

Let's try a simple prompt about food.

In [34]:
## generator setup
generator = pipeline("text-generation")

# response
#response = "I am so sorry to hear that your order was mixed up"

# prompt --> replace new line chars with `.replace`
#prompt = "user: " + text.replace("\n", " ") + " Customer Service Response: " + response

## new prompt
prompt = """
Tell me how to cook pancakes.
"""
## get outputs from text generator
outputs= generator(prompt,
                   max_length=128, #change based on use case
                   )

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [35]:
from IPython.display import Markdown


Markdown(outputs[0]['generated_text'])


Tell me how to cook pancakes.

A couple other people are coming over.

Do I sound like I'm some kind of dumb dork?

It's not funny to be the dumb dork.

I guess I don't have to worry about that too much.

What if I just don't cook pancakes, do I?

It's really hard sometimes.

I feel like I can't even have an egg in my pancake.

My heart rate is so high, I feel like I are going to explode.

I really don't have a problem with

Summary
* The default `gpt-2` model wrote us a poem about pancakes, interesting.
* Let's see if we can improve this with a specific model in the pipeline.

# Let's try using an open source Text Generation model in the pipeline
* We will try a smaller LLM from hugging face.
* This is the model: `openai-community/gpt2-large`
  * model card: https://huggingface.co/openai-community/gpt2-large

In [36]:
## generator setup
generator = pipeline("text-generation",
                     model="openai-community/gpt2-large")

# response
response = "I am so sorry to hear that your order was mixed up"

# prompt --> replace new line chars with `.replace`
#prompt = "user: " + text.replace("\n", " ") + " Customer Service Response: " + response

## new prompt
prompt = """
Tell me how to cook pancakes.
"""
## get outputs from text generator
outputs= generator(prompt,
                   max_length=128, #change based on use case
                   )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [37]:
from IPython.display import Markdown


Markdown(outputs[0]['generated_text'])


Tell me how to cook pancakes.

Pancakes are often made from a combination of flour, eggs, and milk. One of the most important steps in making your own pancake is making sure you get all your flour, eggs, and milk together in a single piece or you will get a wet ball of flour.

To do this, you must first create flour and milk in a blender. Some people use a food processor, but I prefer to use a blender because the flour will separate easier and the batter will become thicker, which makes it easier to mix later.

What are your tips for the proper

Summary
* We now have an output that aligns more with what we actually prompted which was generated text to tell me how to make pancakes.
* Obviously this demonstrates how the pipelines run out of the box but also how there is a lot more that we need to do to make the text generation more specific and actually answer our question or prompt.